In [5]:
#import library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import astropy
import healpy as hp
import scipy.constants as spc
import time
import math

In [6]:
#import data
from astropy.io import fits
#put our own path here
path = "/Users/shiangyilin/21cmCosmology/cosmology/GLEAM_EGC_v2.fits"
hdul = fits.open(path)
df = pd.DataFrame(hdul[1].data)
df['int_flux_076']

0         0.528997
1        -0.032702
2         0.463359
3         2.686571
4         0.393777
            ...   
307450    0.797789
307451    0.719054
307452    0.314990
307453    1.189469
307454   -2.056702
Name: int_flux_076, Length: 307455, dtype: float32

In [14]:
class Catalog():
    
    def __init__(self, df):
        """
        modified from James's code
        Initializes the Catalog and makes a new Pandas DataFrame containing the Right Ascension and Declination as well as all the fluxes
        
        Parameters:
        frame = Pandas DataFrame
        Creates: 
        self.frame as Pandas DataFrame type
        self.RA = right ascencion from J2000 in degrees as numpy.ndarray
        self.DEC = declination from J2000 in degrees as numpy.ndarray
        """
        df.dropna(axis = 1, how = "any")
        df.sort_values(by = "int_flux_wide")
        self.frame = df[[col for col in df.keys() if ((col=='RAJ2000' or col=='DEJ2000'or col[:9]=='int_flux_') 
                                                      and col!= 'int_flux_fit_200')]]
        self.RA = np.asarray(self.frame.RAJ2000)
        self.DEC = np.asarray(self.frame.DEJ2000)
        self.freqMHz = [col[10:] for col in df.keys() if (col[:9]=='int_flux_' and col != 'int_flux_wide')]
        all_time = np.linspace(0, 2*np.pi, 768)
        self.ALTAZ = [data.altaz(t) for t in all_time]
    
    
    def altaz(self, time, rad = False, lat = 37.875*np.pi/180):
        """
        from Max
        Calculates Altitude and Azimuth at given times, centered at HERA
        parameters: 
        time = lst in radian
        ra = Right ascension of the star in degrees or radians
        dec = Declination of the star in degrees or radians
        rad = True if RA and DEC are in radian or False if RA and DEC are in degree
        returns:
        Returns alt, and az all in radian
        """
        ra = self.RA
        dec = self.DEC
        PI = np.pi
        #Setting the latitude/longitude of HERA
        if rad == False:
            ra =ra * PI/180
            dec =dec * PI/180
        
        #Converts the LST to hour angle
        hour_array = np.mod((time - ra), 2*PI)
    
        #Calculates the Altitude and Azimuth
        alt = np.arcsin(np.sin(dec)*np.sin(lat)
                        + np.cos(dec)*np.cos(lat)*np.cos(hour_array))
        az = np.arctan2(np.sin(hour_array)*np.cos(dec),
                        np.cos(hour_array)*np.cos(dec)*np.sin(lat) - np.sin(dec)*np.cos(lat)) + PI
        return [alt, az]
    
    def flux(self, freq, n):
        """return an array of N largest flux sample of frequency FREQ"""
        var = "int_flux_"
        if freq<100:
            var = var + '0' + str(freq)
        else:
            var = var + str(freq)
        print(var)
        return np.asarray(self.frame[var][:n])

In [15]:
data = Catalog(df)
data.frame.head()
data.flux(76, 100)

int_flux_076


array([ 5.28996706e-01, -3.27020250e-02,  4.63359177e-01,  2.68657088e+00,
        3.93777370e-01,  2.15022251e-01,  9.45673063e-02,  3.04107189e-01,
        5.49276769e-01,  1.07047462e+00,  1.15294075e+00, -4.23851330e-03,
        8.62744749e-01,  2.04475626e-01, -8.03497657e-02,  1.90049684e+00,
        7.75041699e-01,  4.59374607e-01,  1.30233908e+00,  2.56895304e-01,
        3.67582023e-01, -1.19067766e-01,  3.80852610e-01,  4.17544007e-01,
        3.92687976e-01,  8.23621929e-01,  5.43193340e-01,  4.19970065e-01,
        1.19083703e+00,  2.28969991e-01,  4.04462636e-01, -1.26657978e-01,
        3.23860943e-01,  8.55139971e-01,  2.90062398e-01, -1.94469169e-01,
        5.18783271e-01,  3.14791232e-01, -4.49459702e-01,  1.12397122e+00,
       -1.28161745e+01,  4.58816826e-01, -3.05185974e-01, -4.86488491e-02,
        1.20394453e-01,  7.02301800e-01,  4.37640309e-01,  4.15691406e-01,
        3.81086320e-01,  9.34650958e-01,  1.06278265e+00,  1.48010030e-01,
        9.48855460e-01,  

In [13]:
all_time = np.linspace(0, 2*np.pi, 768)
data.altaz(all_time[0])

<class 'numpy.float64'>


[array([-0.6555896 , -0.64827305, -0.64660996, ..., -0.17699069,
        -0.5503683 , -1.223605  ], dtype=float32),
 array([3.1418433 , 3.1479514 , 3.1453042 , ..., 0.99152446, 0.4543898 ,
        4.782216  ], dtype=float32)]

In [25]:
class Antenna():
    nside= 32 
    npix = hp.nside2npix(nside)
    mean = 0
    std = 1
    def beam(model):
        if (model == 'one'):
            return 1
        
        elif (model == 'gaussian'):
            def getGaussian(length, mean, std):
                x = np.arange(0,length)
                function = (1/(std*(2*np.pi)**.5))*np.exp(-.5*((x-mean)/std)**2)
                return function
            return getGaussian(npix, mean, std)

        elif (model == "airy"):
            def getAiry(freq, co-altitude, aperture):
                a = aperture
                theta = co-altitude
                j1 = lambda x :
                k = 2 * math.pi * freq /c
                k_a_sin = k * a * math.sin(theta)
                return (2 * J1(k_a_sin) / (k_a_sin))**2
            
            
            
    def phase(b, s):
        assert b.lenght == s.length

SyntaxError: invalid syntax (<ipython-input-25-6e07def041ea>, line 21)

SyntaxError: invalid syntax (<ipython-input-11-94fbaabcbb59>, line 1)